- AI code refactoring and pytest generation tool
- Integrated Qwen/Qwen2.5-Coder-32B-Instruct using the Hugging Face InferenceClient.
- Engineered an XML-structured system prompt to ensure the model outputs parseable refactored code and test suites.
- Built a custom Gradio UI with dedicated tabs for viewing the cleaned code and the generated Pytest unit tests.

In [ ]:
!pip install -q huggingface_hub gradio

In [ ]:
import os
import gradio as gr
from huggingface_hub import InferenceClient

In [ ]:
client = InferenceClient(
    model="Qwen/Qwen2.5-Coder-32B-Instruct",
    token=os.environ.get("HF_TOKEN")
)

In [ ]:
def clean_and_test_code(messy_code):
    if not messy_code.strip():
        return "Please paste some code first!", "No tests to generate."
        
    # 1. The System Prompt with strict XML formatting rules
    system_prompt = """You are an expert Senior Python Engineer. 
    The user will provide you with messy or undocumented Python code. 
    Your job is to:
    1. Refactor the code to be clean, readable, and efficient.
    2. Add Google-style docstrings, type hints, and helpful inline comments.
    3. Write a comprehensive suite of `pytest` unit tests for the code, including edge cases.
    
    You MUST return ONLY the code, wrapped in specific XML tags exactly like this:
    <refactored>
    def your_clean_function():
        pass
    </refactored>
    <tests>
    def test_your_clean_function():
        assert True
    </tests>
    
    Do not include markdown formatting like ```python inside the tags. Just the raw code.
    """
    
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": messy_code}
    ]
    
    try:
        # 2. Call the Qwen-Coder model
        response = client.chat_completion(
            messages=messages,
            max_tokens=2000,
            temperature=0.2 # Low temperature for more deterministic, reliable code
        )
        
        full_text = response.choices[0].message.content
        
        # 3. Parse the XML tags
        refactored_code = "Error parsing refactored code."
        test_code = "Error parsing test code."
        
        if "<refactored>" in full_text and "</refactored>" in full_text:
            refactored_code = full_text.split("<refactored>")[1].split("</refactored>")[0].strip()
            
        if "<tests>" in full_text and "</tests>" in full_text:
            test_code = full_text.split("<tests>")[1].split("</tests>")[0].strip()
            
        # Clean up accidental markdown backticks if the model ignores instructions
        refactored_code = refactored_code.replace("```python", "").replace("```", "").strip()
        test_code = test_code.replace("```python", "").replace("```", "").strip()
            
        return refactored_code, test_code
        
    except Exception as e:
        return f"# Error analyzing code:\n{str(e)}", ""

In [ ]:
with gr.Blocks(theme=gr.themes.Monochrome()) as demo:
    gr.Markdown("#Clean Code & Unit Test Assistant")
    gr.Markdown("Paste your undocumented or messy Python code below. The AI will refactor it, add docstrings, and generate a complete suite of Pytest unit tests.")
    
    with gr.Row():
        # Left side: Input
        with gr.Column(scale=1):
            input_code = gr.Code(
                label="Paste Messy Code Here", 
                language="python", 
                lines=15
            )
            submit_btn = gr.Button("Clean My Code", variant="primary")
            
        # Right side: Outputs
        with gr.Column(scale=1):
            with gr.Tabs():
                with gr.TabItem("Refactored Code"):
                    output_refactored = gr.Code(label="Clean Code with Docstrings", language="python", interactive=False)
                with gr.TabItem("Pytest Unit Tests"):
                    output_tests = gr.Code(label="Test Suite", language="python", interactive=False)

    # 4. Wire the button to the function using chained events for a loading state
    submit_btn.click(
        fn=lambda: gr.update(value="Analyzing code...", interactive=False),
        inputs=[],
        outputs=[submit_btn]
    ).then(
        fn=clean_and_test_code,
        inputs=[input_code],
        outputs=[output_refactored, output_tests]
    ).then(
        fn=lambda: gr.update(value="Clean My Code", interactive=True),
        inputs=[],
        outputs=[submit_btn]
    )

In [ ]:
demo.launch()